In [1]:
import os
from my_secrets import base_path

paths_dict=  {
    "ochiai_scores_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiScores"),
    "ranking_files_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiRankings"),
    "data_file_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
    "output_metrics_per_bug_file": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiResultsMetricsPerBug.csv"),
    "output_metrics_per_project_file": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiResultsMetricsPerProject.csv")
}

In [7]:
import glob
import os
import csv
import utils
import importlib
importlib.reload(utils)
top_k = [1, 3, 5, 10]
top_k_obj = {}

bugs_data = utils.json_file_to_dict(paths_dict["data_file_path"])
bug_metrics = {}
project_metrics = {}

ochiai_scores_folders_list = utils.get_folder_names(paths_dict["ochiai_scores_path"])

for ochiai_output_folder in ochiai_scores_folders_list:
    folders = ochiai_output_folder.split(os.sep)
    folders = [i for i in folders if i]
    ochiai_identificator= folders[-1].replace("/", "").replace("\/", "")

    top_k_obj [ochiai_identificator] = {}

    ochiai_scores_path = os.path.join(paths_dict["ochiai_scores_path"], ochiai_output_folder)
    failing_tests_file_name = utils.get_json_files(ochiai_scores_path)
    fake_failing_tests_info = utils.json_file_to_dict(failing_tests_file_name[0])
    for bug_report_analysis_file in glob.glob(ochiai_scores_path + os.sep + "*" + os.sep + "*.json"):

        project = bug_report_analysis_file.split(os.sep)[-2]

        bug_id = bug_report_analysis_file.split(os.sep)[-1].replace(".json","")

        print("---------------")
        print(ochiai_identificator + " - " + project + " - " + bug_id)
        if "buggyMethods" not in bugs_data[project][bug_id].keys() or bugs_data[project][bug_id]["buggyMethods"] == {}:
            print("No buggy methods. Skipping it")
            continue
        buggyMethods = bugs_data[project][bug_id]["buggyMethods"]
        stack_trace_files = bugs_data[project][bug_id]["stack_trace_files"]
        stack_trace_methods_not_formatted = bugs_data[project][bug_id]["stack_trace_methods"]
        stack_trace_methods = []
        for method in stack_trace_methods_not_formatted:
            method_formatted = method
            if "$" in method_formatted:
                method_formatted = utils.remove_between_dollar_and_dot(method_formatted)
            stack_trace_methods.append(method_formatted)
        ochiai_scores_data = utils.json_file_to_dict(bug_report_analysis_file)
        ochiai_scores_data = utils.sort_dict_by_values_reverse_order(ochiai_scores_data)
        if len(ochiai_scores_data) == 0:
            print("No Ochiai scores for this bug due to gzoltar files problem. Skipping it")
            continue

        ranking = utils.rank_methods(ochiai_scores_data)

        ranking_file_name = os.path.join(paths_dict["ranking_files_path"], ochiai_identificator, project, bug_id + ".json")
        utils.dict_to_json_file(ranking_file_name, ranking)

        buggy_methods_list = utils.extract_buggy_methods_list (ranking, buggyMethods)
        st_ranking = utils.get_st_raking_dict(stack_trace_methods)

        N = 10 # TODO - Modify if necessary
        if project not in bug_metrics.keys():
            bug_metrics[project] = {}
        if bug_id not in bug_metrics[project].keys():
            bug_metrics[project][bug_id] = {}
        bug_obj = bug_metrics[project][bug_id]
        if project not in fake_failing_tests_info.keys() or bug_id not in fake_failing_tests_info[project].keys():
            continue
        if 'Stack Trace (ST) size' not in bug_obj.keys():
            bug_obj['Stack Trace (ST) size'] = len(stack_trace_files)
            number_buggy_methods = 0
            for file in buggyMethods.keys():
                number_buggy_methods += len(buggyMethods[file])
            bug_obj['Number of buggy methods'] = number_buggy_methods
            bug_obj['Position of the first buggy method into the ST'] = utils.get_first_buggy_method_in_stack_trace(buggy_methods_list, stack_trace_methods)
            bug_obj['Precision ST Top 10'] = utils.get_precision_top_n (st_ranking, N, buggy_methods_list )
            bug_obj['Recall ST Top 10'] = utils.get_recall_top_n (st_ranking, N, buggy_methods_list )
            bug_obj['F1 ST Top 10'] = utils.get_f1_top_n (st_ranking, N, buggy_methods_list )
            if 'Stack Trace' not in top_k_obj.keys():
                top_k_obj ['Stack Trace'] = {}
            for k in top_k:
                if f"Top {k}" not in top_k_obj ['Stack Trace'].keys():
                    top_k_obj ['Stack Trace'][f"Top {k}"] = 0
                try:
                    pos = int(bug_obj['Position of the first buggy method into the ST'])
                except ValueError:
                    continue
                if pos <= k:
                    top_k_obj ['Stack Trace'][f"Top {k}"] += 1


        bug_obj['Position of the first buggy method into the ' + ochiai_identificator] = utils.get_best_classified_buggy_method (ranking, buggy_methods_list)
        bug_obj['Precision ' + ochiai_identificator + ' Top 10'] = utils.get_precision_top_n (ranking, N, buggy_methods_list )
        bug_obj['Recall ' + ochiai_identificator + ' Top 10'] = utils.get_recall_top_n (ranking, N, buggy_methods_list )
        bug_obj['F1 ' + ochiai_identificator + ' Top 10'] = utils.get_f1_top_n (ranking, N, buggy_methods_list )
        for k in top_k:
            if f"Top {k}" not in top_k_obj [ochiai_identificator].keys():
                top_k_obj [ochiai_identificator][f"Top {k}"] = 0
            try:
                pos = int(bug_obj['Position of the first buggy method into the ' + ochiai_identificator])
            except ValueError:
                continue
            if pos <=k:
                top_k_obj [ochiai_identificator][f"Top {k}"] += 1
        if ochiai_identificator == "originalOchiai":
            bug_obj['Number of fake failing tests '+ ochiai_identificator] =  fake_failing_tests_info[project][bug_id]["failing_tests_number"]
            bug_obj['Number of fake passing tests '+ ochiai_identificator] =  fake_failing_tests_info[project][bug_id]["passing_tests_number"]
        else:
            bug_obj['Number of fake failing tests '+ ochiai_identificator] =  fake_failing_tests_info[project][bug_id]["fake_failing_tests_number"]
            bug_obj['Number of fake passing tests '+ ochiai_identificator] =  fake_failing_tests_info[project][bug_id]["fake_passing_tests_number"]
        print("Number of buggy methods - " + str(len(buggy_methods_list)))
        print("Stack trace size - " + str(N))
        print("Position of the first buggy method in the stack trace - " + str(bug_obj['Position of the first buggy method into the ST']))
        print("Position of the first buggy method into the Ochiai classification - " + str(bug_obj['Position of the first buggy method into the ' + ochiai_identificator]))
        print("Precision Ochiai Top " + str(N) + " - " + str(bug_obj['Precision ' + ochiai_identificator + ' Top 10']))
        print("Recall Ochiai Top " + str(N) + " - " + str(bug_obj['Recall ' + ochiai_identificator + ' Top 10']))
        print("F1 Ochiai Top " + str(N) + " - " + str(bug_obj['F1 ' + ochiai_identificator + ' Top 10']))
        bug_metrics[project][bug_id] = bug_obj

    print("")
    print("")
    print("======= " + ochiai_identificator + " =======")
    for project in bugs_data.keys():
        if project not in project_metrics.keys():
            project_metrics[project] = {}
        proj_obj = project_metrics[project]
        if project not in fake_failing_tests_info.keys() :
            continue

        if 'Map  Stack Traces' not in proj_obj.keys():
            proj_obj['Map Stack Traces'] = utils.get_map (project, "stackTraces",bugs_data[project], None)
            proj_obj['MRR Stack Traces'] = utils.get_mrr (project, "stackTraces",bugs_data[project], None)

        proj_obj['Map ' + ochiai_identificator] = utils.get_map (project, ochiai_identificator, bugs_data[project], paths_dict["ranking_files_path"])
        proj_obj['MRR ' + ochiai_identificator] = utils.get_mrr (project, ochiai_identificator,bugs_data[project], paths_dict["ranking_files_path"])
        project_metrics[project] = proj_obj

        print("")
        print("----")
        print(project)
        print(f"Map {ochiai_identificator} - " + str(proj_obj['Map ' + ochiai_identificator]))
        print(f"MRR {ochiai_identificator}- " + str(proj_obj['MRR ' + ochiai_identificator]))
    print("==============")
    print("")

utils.create_or_update_bug_metrics_file(bug_metrics, paths_dict["output_metrics_per_bug_file"], ochiai_scores_folders_list)
utils.create_or_update_project_metrics_file(project_metrics,  paths_dict["output_metrics_per_project_file"], ochiai_scores_folders_list)

with open('../data/top_k_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    headers = ["Técnica"] + list(top_k_obj[next(iter(top_k_obj))].keys())
    writer.writerow(headers)
    for technique, values in top_k_obj.items():
        writer.writerow([technique] + list(values.values()))

print("Done")


---------------
modifiedOchiai1.1 - Closure - 37
Number of buggy methods - 2
Stack trace size - 10
Position of the first buggy method in the stack trace - 2
Position of the first buggy method into the Ochiai classification - 298
Precision Ochiai Top 10 - 0.0
Recall Ochiai Top 10 - 0.0
F1 Ochiai Top 10 - 0.0
---------------
modifiedOchiai1.1 - Closure - 125
Number of buggy methods - 2
Stack trace size - 10
Position of the first buggy method in the stack trace - 9
Position of the first buggy method into the Ochiai classification - 1114
Precision Ochiai Top 10 - 0.0
Recall Ochiai Top 10 - 0.0
F1 Ochiai Top 10 - 0.0
---------------
modifiedOchiai1.1 - Closure - 152
Number of buggy methods - 2
Stack trace size - 10
Position of the first buggy method in the stack trace - 8
Position of the first buggy method into the Ochiai classification - 8
Precision Ochiai Top 10 - 0.1
Recall Ochiai Top 10 - 0.5
F1 Ochiai Top 10 - 0.16666666666666669
---------------
modifiedOchiai1.1 - Closure - 2
Number o